# Język Python
## Wątki i procesy

https://docs.python.org/3/library/threading.html

A programmer had a problem.

"I know! I will solve it with threads!" - he thought to himself.

has Now problems. two he

### Tworzenie i uruchamianie wątku

In [ ]:
import threading

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            if i % 1e4 == 0:
                print(i)
                
t1 = Thread1()
t2 = Thread1()

t1.start()
t2.start()
print("Finished")

In [ ]:
p = threading.Thread(target=print, args=('bob', 'john'), kwargs={'sep': '\t'})
p.start()

A gdyby tak:

In [ ]:
@threaded
def f():
    pass

f()

### Synchronizacja wątków

#### Lock

In [ ]:
import threading
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(threading.Thread):
    def run(self):
        running = True
        while running:
            lock.acquire()
            if not queue:
                running = False
            else:
                elem = queue.pop()
            lock.release()
            time.sleep(2) # do something with elem
            lock.acquire()
            print(self.name, elem)
            lock.release()
            
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()
t1.join()
t2.join()
print("Finished")

In [ ]:
import threading
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(threading.Thread):
    def run(self):
        while True:
            with lock:
                if not queue:
                    break
                else:
                    elem = queue.pop()
            time.sleep(3) # do something with elem
            with lock:
                print(self.name, elem)
            
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()
t1.join()
t2.join()
print("Finished")

`Lock`:
* może zostać zwolniony przez **dowolny wątek**
* próba ponownego zajęcia przez ten sam wątek blokuje go *ad infinitum*
* nie można zwolnić, jeżeli nie został zajęty
* można spróbować zająć w trybie nieblokującym
* nie jest automatycznie zwalniany

Polecam `RLock`:
* może zostać zwolniony tylko przez wątek, który go posiada
* można go zajmować wielokrotnie, ale należy zwolnić tyle samo razy
* jest zwalniany automatycznie po zakończeniu wątku

A gdyby tak:

In [ ]:
@synchronized
def f():
    pass

In [ ]:
type(threading.Lock())

In [ ]:
import threading
import time

lock = threading.RLock()

class Thread1(threading.Thread):
    def run(self, *args):
        print(lock.acquire(False))
#         print(lock.acquire(timeout=3))  # alt
        time.sleep(1)
        lock.release()
        
                
Thread1().start()
# time.sleep(2)
Thread1().start()

#### Condition

In [ ]:
import threading
import time

cv = threading.Condition()

l = []

class Consumer(threading.Thread):
    def run(self):
        '''Consume one item'''
        with cv:
            while True:
                while not l:
                    cv.wait()
                print(l.pop(0))

class Producer(threading.Thread):
    def run(self):
        '''Produce one item'''
        global l
        for i in range(10):
            with cv:
                l += [i]
                cv.notify()
            time.sleep(1)

Consumer().start()
time.sleep(2)
Producer().start()

In [ ]:
print(l)

#### Inne

* `Semaphore`
* `BoundedSemaphore`
* `Event`
* `Timer`
* `Barrier`

### Wydajność

In [ ]:
import threading
import time

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.start()
t2.start()
print("Finished")

t1.join()
t2.join()

finish = time.time()
print(finish - start)

In [ ]:
import threading
import time

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.run()
t2.run()
print("Finished")

finish = time.time()
print(finish - start)

## Procesy

https://docs.python.org/3/library/multiprocessing.html

### Wydajność

In [ ]:
import multiprocessing
import time

class Thread1(multiprocessing.Process):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.run()
t2.run()
print("Finished")

finish = time.time()
print(finish - start)

In [ ]:
import multiprocessing
import time

class Thread1(multiprocessing.Process):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.start()
t2.start()
print("Finished")

t1.join()
t2.join()
finish = time.time()
print(finish - start)

Procesy są cięższe i mniej wygodne w użyciu, ale omijają GIL.

In [ ]:
import multiprocessing
import time

queue = list(range(10))
lock = multiprocessing.Lock()

class Consumer(multiprocessing.Process):
    def run(self):
        while True:
            with lock:
                if not queue:
                    break
                else:
                    elem = queue.pop()
            time.sleep(2) # do something with elem
            with lock:
                print(self.pid, elem)
                        
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()

Przydatne klasy:
- Lock
- Queue
- JoinableQueue
- Pipe
- Value
- Array